In [1]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# StyleGAN3 from NVIDIA

**Notes**
* To see the original code from NVIDIA [Check here](https://github.com/NVlabs/stylegan3)
* We are using a pretrained model and fine-tuning on top of it.
* If you come across bugs please post them in [Discord](https://discord.com/invite/awREd7EtMA)

---

If you find this notebook useful, consider signing up for my [Code Sprout Newsletter](https://codesprout.substack.com/welcome) or [Check my links](https://shyambv.bio.link/)

Medium article related to it is mentioned []()

# Setup

In [2]:
# originally was: torch==1.11.0+cu113 torchvision==0.11.1+cu113
!pip install torch==1.13.0
# torchvision torchaudio -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.0 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torc

In [3]:
!pip install einops ninja gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 11.7 MB/s eta 0:00:00


In [4]:
import os
if os.path.isdir('/content/drive/MyDrive/WIP/stylegan3/'):
    %cd '/content/drive/MyDrive/WIP/stylegan3/'
else:
    !git clone https://github.com/NVLabs/stylegan3.git /content/drive/MyDrive/WIP/stylegan3/
    %cd '/content/drive/MyDrive/WIP/stylegan3/'


/content/drive/MyDrive/WIP/stylegan3


# Making a Dataset

In [7]:
!python /content/drive/MyDrive/WIP/stylegan3/dataset_tool.py \
    --source=/content/drive/MyDrive/dev/arco/240523_01-rose01-YFlips-6min \
    --dest=/content/drive/MyDrive/dev/arco/240523_01-rose01-YFlips-6min.zip \
    --resolution='1024x1024'

100% 10800/10800 [19:14<00:00,  9.36it/s]


# Model training

You can start from a pre-trained model. Below are some of the models from Nvdia



```
stylegan3-t-ffhq-1024x1024.pkl, stylegan3-t-ffhqu-1024x1024.pkl, stylegan3-t-ffhqu-256x256.pkl
stylegan3-r-ffhq-1024x1024.pkl, stylegan3-r-ffhqu-1024x1024.pkl, stylegan3-r-ffhqu-256x256.pkl
stylegan3-t-metfaces-1024x1024.pkl, stylegan3-t-metfacesu-1024x1024.pkl
stylegan3-r-metfaces-1024x1024.pkl, stylegan3-r-metfacesu-1024x1024.pkl
stylegan3-t-afhqv2-512x512.pkl
stylegan3-r-afhqv2-512x512.pkl
```



## Using pre-trainined model

In [ ]:
# Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
# !python /content/drive/MyDrive/WIP/stylegan3/train.py --help
resume_from = '/content/drive/MyDrive/dev/pretrained-models/stylegan3-r-ffhqu-1024x1024.pkl'
# resume_from = '/content/drive/MyDrive/dev/arco/training/00020-stylegan3-r-rose01-blobs-square-gpus1-batch8-gamma6.6/network-snapshot-000040.pkl'
!python /content/drive/MyDrive/WIP/stylegan3/train.py \
    --outdir=/content/drive/MyDrive/dev/arco/training \
    --cfg=stylegan3-r \
    --data /content/drive/MyDrive/dev/arco/240523_01-rose01-YFlips-6min.zip \
    --gpus=1 --batch=8 --batch-gpu=4 --gamma=6.6 --mirror=1 --kimg=500 --snap=3 \
    --resume {resume_from}



Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9997227795604651,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_si